In [58]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

In [37]:
from langchain_openai import ChatOpenAI

In [38]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo-0125')

Cargar la información

In [39]:
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ('post-content', 'post-title', 'post-header')
        )
    ),
)

In [40]:
docs = loader.load()

In [41]:
len(docs[0].page_content)

42823

In [45]:
print(docs[0].page_content[:600])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several 


In [47]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

Chunks de información y guardar en un vectorstore (Chroma DB)

In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=200,
                                               add_start_index=True)

In [49]:
splits  = text_splitter.split_documents(docs)

In [50]:
len(splits)

66

In [54]:
len(splits[1].page_content)

609

In [55]:
splits[1].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 971}

In [56]:
splits[1].page_content

'Memory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.'

In [57]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

Retrieve and generate

In [60]:
retriever = vectorstore.as_retriever()

In [59]:
#prompt = hub.pull('rlm/rag-prompt')

In [66]:
template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Always reply in Spanish.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [67]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [68]:
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [31]:
rag_chain.invoke('que es una descomposicion')

'Una descomposición es el proceso de dividir un problema en pasos más pequeños y manejables para abordarlo de manera más efectiva. En el contexto de la técnica CoT, implica la generación de múltiples pensamientos por paso para crear una estructura en forma de árbol. Esto puede llevarse a cabo utilizando instrucciones específicas para tareas o mediante la interacción con humanos.'

In [69]:
rag_chain.invoke('que es la tecnica CoT')

'CoT es una técnica de promoción estándar para mejorar el rendimiento del modelo en tareas complejas al descomponer las tareas difíciles en pasos más pequeños y simples. Utiliza la instrucción de "pensar paso a paso" para aprovechar más la computación en tiempo de prueba. Gracias por preguntar!'

In [64]:
rag_chain.invoke('En que epoca vivieron los mayas')

'Los mayas vivieron en la época precolombina, especialmente en Mesoamérica, durante la época clásica que abarcó desde aproximadamente el año 250 d.C. hasta el 900 d.C. Gracias por preguntar!'

In [70]:
rag_chain.invoke('quiero dictar una charla, haz un indice para los tiutulos de los temas a tratar, no mas de 5 titulos')

'Para dictar una charla, los títulos de los temas a tratar podrían ser:\n\n1. Introducción al tema\n2. Desarrollo de la charla\n3. Ejemplos prácticos\n4. Conclusiones\n5. Preguntas y respuestas\n\n¡Gracias por preguntar!'

In [71]:
vectorstore.delete_collection()